# Organized data

1. 上传数据
* 连接拷回来的光盘，然后利用ftp软件连接porter之后，上传数据。
* 所有的原始数据，都放到 /home/data/rawdata 文件夹下，每一个session用一个文件夹表示，通常为 ‘年月日_实验缩写_被试代号’的格式，例如“20220110_CET_CN003”
* 在该文件夹下，新建一个 `mat_files_from_exp`的文件夹用来存放所有的scan里面的行为数据

2.  打开terminal, 开放权限让别人也能读取

In [ ]:
cd /home/data/rawdata/CET_fmri/20221102_CET_CN016/
! chgrp -R student 20221102_CET_CN016 #修改该文件夹的所属组别
! chmod g+w 20221102_CET_CN016 #修改该文件夹的权限

3. 整理数据  

In [ ]:
! dcm2niix_afni -f %t_%s_%p rawdata/
# afni函数dcm2niix_afni -[filename] time_seriesnumber_protocol
# output: .nii文件是nifit格式MRI数据文件，.json文件是对应的参数说明文件

# 整理无关数据：原始的dicom文件，localizer，以及费的结构或者没扫完的功能像等
! cd rawfiles/
! mkdir rawfiles
! mv *localizer* rawfiles
! mv *Localizer* rawfiles
! mv *Dicom* rawfiles
# 一般的fMRI实验只需要留下t1、epi和field-map的数据

# Anatprep

In [ ]:
# 设置一些variable
import os
t1 = '20221209111810_6_t1_mpr_sag_iso_mww.nii'  # set the t1 file
subj = 'CN022' # 设定一个variable

t1_2FS = f'{subj}_2FS.nii.gz' 
AFNI_HOME = os.getenv('AFNIDIR')
FREESURFER_HOME = os.getenv('FREESURFER_HOME') # 获得 FreeSurfer的主文件夹路径
MNIT1 = AFNI_HOME + '/MNI152_2009_template_SSW.nii.gz' # 我们用MNI152去掉头皮的模板作为我们的base


%cd ../ # 我们先走到数据文件夹
! mkdir anatpp # 新建一个文件夹用来处理t1像
! cp rawdata/{t1} anatpp/ # 把t1像拷贝一份到文件夹里面
%cd anatpp/

1. step1: 对t1像进行简单处理  
     
   得到 {t1_2FS}

In [ ]:
#### 改变nifti头文件信息，进行标准化，不修改数据
# 把图像的中点划到统一显示空间的中点
! 3drefit -deoblique -xorigin cen -yorigin cen -zorigin cen {t1}
# 把图像的朝向划到和标准朝向一致,这一步生成t12std.nii.gz的文件
! fslreorient2std {t1} t1_2std.nii.gz 


# 简单的剥头皮
! 3dSkullStrip -input t1_2std.nii.gz -prefix t1_2std_ss.nii.gz -orig_vol

# 上面一步把剥掉头皮的t1像和MNI模板按重心位置移动，同时移动t12std.nii.gz, 这样做是为了让其和MNI模板空间上更一致。
! @Align_Centers -cm -base {MNIT1} -dset t1_2std_ss.nii.gz -child t1_2std.nii.gz


# 清除一下
! mv t1_2std_shft.nii.gz {t1_2FS} 
! rm *t1_2std* # 删除掉多余文件

# 先检查/home/data/subjectT1/{subj} 是否存在，如果没有，创建一个文件夹
! mkdir /home/data/subjectT1/{subj}
# 把FreeSurfer的T1文件拷贝进去
! cp {t1_2FS} /home/data/subjectT1/{subj}/


这一步需要利用fsleyes来人工确认（在跑FreeSurfer之前）

2. step2: 把{t1_2FS}送给freesurfer进行皮层重建  
     
    recon-all 会在/home/software/freesurfer/subjects下面创建一个{subj}名字的文件夹，例如CN003。如果该文件夹已经存在(例如recon-all中途中断，但是重新跑一次)都会报错。这时删除之前的旧文件夹就行了。所有的报错信息都在reconlog.txt这个文件下。

    这一步可能会要很长时间。在stout上面测试跑单个被试需要6个小时。。这一步所有的输出都在reconlog.txt里面
    这一步跑完之后。把非线性配准的结果文件移动到FreeSurfer被试文件夹下面

In [ ]:
cd /home/data/subjectT1/{subj}
! recon-all -s {subj} -i {subj}_2FS.nii.gz -all > ./reconlog.txt

完成之后检查：  
/home/software/freesurfer/7.3.2/subjects/{subj}/mri  
* ***brainmask.mgz***  
* ***T1.mgz***

freeview:  
***brainmask.nii.gz***  
***lh_pial***, ***rh_pial***  
***lh_white***,  ***rh_white***  

3. step3: 将t1结构像线性和非线性配准到mni空间  
     
    将个体的t1像配准到mni标准空间和mni152的标准模板，这是基于volume的分析的基础
   可以和step2平行跑。

In [ ]:
# 重开一个terminal，重新设置变量

cd /home/data/subjectT1/{subj}

import os
subj = 'CN022' # 设定一个variable
t1_2FS = f'{subj}_2FS.nii.gz' 
AFNI_HOME = os.getenv('AFNIDIR')
FREESURFER_HOME = os.getenv('FREESURFER_HOME') # 获得 FreeSurfer的主文件夹路径
MNIT1 = AFNI_HOME + '/MNI152_2009_template_SSW.nii.gz' # 我们用MNI152去掉头皮的模板作为我们的base


#cd /home/data/subjectT1/{subj}
! mkdir mni # make a new directory
! @SSwarper -input {t1_2FS} -base {MNIT1} -subid {subj} -odir mni -verb

# 创建一个剥头皮的mask，如果剥头皮效果不满意，可以手动编辑这个mask，然后再手动剥头皮
%cd mni/
! 3dcalc -a anatSS.{subj}.nii -expr 'step(a)' -prefix anatSSmask.{subj}.nii
print('Nonlinear warping done!')

利用afni做基于volume的非线性配准，完成之后需要人工检查配准结果  
* ***anatSS.{sub}.nii***

4. step4: afni中皮层加工模块SUMA的设置

In [ ]:
# 把step3中间的非线性配准的文件夹放到freesurfer里面去
! mv /home/data/subjectT1/{subj}/mni {FREESURFER_HOME}/subjects/{subj}/ # 移动step3获得到mni文件夹到step2生成的文件夹

% cd /home/software/freesurfer/7.3.2/subjects/{subj}/
# AFNI中用来进行皮层加工模块的是SUMA。利用FreeSurfer进行了皮层重建之后，需要对SUMA进行一定的设置
! @SUMA_Make_Spec_FS -sid {subj} -fspath {FREESURFER_HOME}/subjects/{subj} -NIFTI > SUMA_Make_Spec_FS.log

以上的命令会在FreeSurfer subjects中再创建一个SUMA的文件夹，里面包含了SUMA需要的一些文件。SUMA是连接AFNI和FreeSurfer之间的工具，让在AFNI上进行皮层分析成为可能。

这一步可能需要10分钟左右。完成之后，我们打开远程桌面，打开一个terminal (注意这一步是bash命令，不用ipython)，输入以下的命令，可以打开afni和suma来检查互相的联通情况。

In [ ]:
# 注意替换被试名
cd $FREESURFER_HOME/subjects/CN003/SUMA
afni -niml & suma -spec CN003_both.spec -sv CN003_SurfVol.nii

# Funcprep

In [ ]:
from RZutilpy.system import Path, unix_wrapper, gettimestr, makedirs
from RZutilpy.rzio import matchfiles
from RZutilpy.figure import plot
from RZutilpy.mri import findminoutlier
import os

# subject name
subj = 'CN026'

# a list of functional NIFTI files
files = matchfiles(f'/home/data/rawdata/CET_fmri/20221208_CET_CN026/rawdata/*epi*.nii')

# t1 file
t1 = Path(f'/home/software/freesurfer/7.3.2/subjects/{subj}/SUMA/{subj}_SurfVol.nii')
t1ss = Path(f'/home/software/freesurfer/7.3.2/subjects/{subj}/mni/anatSS.{subj}.nii')

'''
###################如果是中途回来重新设置
t1 = Path(f'{t1.pstem}+orig')  # switch t1 to the new location
FREESURFER_HOME = os.getenv('FREESURFER_HOME')
AFNI_HOME = os.getenv('AFNIDIR')
cwd = Path.cwd() # record current directory, we will go back
orig_shell = os.environ['SHELL']
'''

# output directory, if exists, we exit
output_dir = Path(f'/home/data/rawdata/CET_fmri/20221208_CET_CN026/funcpp/')

# number of tr to discard
tr_discard = 0  # number of tr to discard

# motion censor limit
motion_censor = 0.3  # threshold for motion censoring, default:(0.3)

# extra option for align_epi_anat.py
align_opt = ['-giant_move']

fwdfiles = [ '01', '03','05','07'] # 这里需要手动run的顺序，用来做配对distortion correction
revfiles = ['02','04','06','08']

# some calculation
nRuns = len(files)
runstr = [f'{i+1:02d}' for i in range(nRuns)]############################################################
motion_censor = 0.3 if motion_censor is None else motion_censor

## funcpp01


* 检查文件，设置文件，复制文件  

In [ ]:
# 检查文件
# ============== deal with some input parameters =======================
# Show some diagnostic information
# check file exists

#cd rawdata/

for i in files:
    assert Path(i).exists(), f'{i} does not exist!'

t1 = Path(t1)
assert t1.exists(), 'T1 file does not exist!'



# generate slice timing file
if not Path('SliceTiming.txt').exists():
    import json 
    import numpy as np
    with open(f'{Path(files[0]).pstem}.json') as f:
        jsoninfo = json.load(f)
    np.savetxt('SliceTiming.txt', jsoninfo['SliceTiming'])
    print('generate SliceTiming.txt!')
    del jsoninfo

# 整理forward和reverse的文件，如果两者数量不相等，我们自动补齐缺数量
if len(fwdfiles)!=len(fwdfiles):
    if len(fwdfiles)>len(revfiles):
        nPair = len(fwdfiles)
        revfiles = revfiles + [revfiles[end]]*(nPair-len(revfiles))
    else:
        nPair = len(revfiles)
        fwdfiles = fwdfiles + [fwdfiles[end]]*(nPair-len(fwdfiles))

# print out some diagnoistic
print(f'\nt1 file is: \n{t1}')
print(f'functional files are: \n')
[print(f'{i}') for i in files]
print(f'\nforward files are: {fwdfiles} \n')
print(f'reverse files are: {revfiles} \n')
print(f'\nslice timing file is: \nSliceTiming.txt')

In [ ]:
# 进一步设置
start_time = gettimestr("full")
print(f'\n=============== Preprocessing started: {start_time} ================\n')
# change shell to tcsh, afni default shell is tcsh
orig_shell = os.environ['SHELL']
os.environ['SHELL']='/usr/bin/tcsh'
cwd = Path.cwd() # record current directory, we will go back

# verify that the results directory does not yet exist
output_dir = Path(output_dir)
assert not output_dir.exists(), f'output dir {output_dir} already exists'
makedirs(output_dir)
#makedirs((output_dir / 'stimuli'))
# enter the results directory (can begin processing data)
os.chdir(output_dir)
# copy anatomy to results dir
! 3dcopy {t1} {t1.pstem}
t1 = Path(f'{t1.pstem}+orig')  # switch t1 to the new location

# copy nonlinear-warping MNI files to this folder
FREESURFER_HOME = os.getenv('FREESURFER_HOME')
AFNI_HOME = os.getenv('AFNIDIR')




! cp {FREESURFER_HOME}/subjects/{subj}/mni/anatQQ.{subj}_WARP.nii ./
! cp {FREESURFER_HOME}/subjects/{subj}/mni/anatQQ.{subj}.aff12.1D ./
! cp {AFNI_HOME}/MNI152_2009_template_SSW.nii.gz ./

# 利用fslreorient2std把朝向划到和MNI一样
for file in files:
    ! fslreorient2std {file}

* 把所有的功能像数据先copy到一个<output_dir>文件夹下, 避免修改原始的功能像数据, 并且去掉每个run前面的几个TR <tr_discard>  
  **----pb00.{subj}.r{run}.tcat**

In [ ]:

# 复制文件到<output_dir>，同时去掉最开始几个TR
# ============================ auto block: tcat ============================
# apply 3dTcat to copy input dsets to results dir,
# while might want removing the first TRs
for file, run in zip(files, runstr):
    cmd=f'3dTcat -prefix pb00.{subj}.r{run}.tcat {file}[{tr_discard}..$]'
    unix_wrapper(cmd)

 * 矫正epi的朝向和正中点  
   其中如果epi和T1是在不同的session采集的，那么可能会有比较大的偏差。需要手动的移动epi的图像到和用来做FreeSurfer的t1像一致  
  **----pb00.{subj}.r{run}.tcat+orig**


In [ ]:
# 移动EPI文件，和T1中心重合
# ============================auto block: giant_move, added by RZ ===========================
for run in runstr:
    # 把图像的中点划到统一显示空间的中点
    ! 3drefit -deoblique -xorigin cen -yorigin cen -zorigin cen pb00.{subj}.r{run}.tcat+orig
    # 然后把function数据和 t1ss文件的重心移动到一致，这样有利于进行配准
    ! @Align_Centers -cm -no_cp -base {t1ss} -dset pb00.{subj}.r{run}.tcat+orig

! rm *_shft.1D

 * 到<output_dir> 文件夹下面，找到头动最小的volume，然后把这个作为头动矫正的基准。这一步会检查头动，并且记录一些特别大头动的volume，以头动超过一定的threshold为准

In [ ]:
# 同时找到outlier最小的一个volume作为后面motion correction的base
findminoutlier(f'pb00.{subj}.r*.tcat+orig.HEAD', output_dir)
# this part will generate several files
#   out.pre_ss_warn.txt: warning for pre-steady state in the first TRs, consider change tr_discard
#   outcont.r**.1D: fraction of outlier in each volume
#   outcount_rall.1D concatenate fraction of outlier
#   out.min_outlier.txt  tells you which run, which TR is min_outlier

## funcpp02

### 1. step1: Despike

In [ ]:
# ================================ despike =================================
for run in runstr:
    ! 3dDespike -NEW -nomask -prefix pb01.{subj}.r{run}.despike pb00.{subj}.r{run}.tcat+orig
    
#! rm pb00*

### 2. step 2: slice-timing correction

In [ ]:
# ================================= tshift =================================
for run in runstr:
    ! 3dTshift -tzero 0 -quintic -prefix pb02.{subj}.r{run}.tshift \
         -verbose -tpattern @{'../rawdata/SliceTiming.txt'} pb01.{subj}.r{run}.despike+orig

# 删掉上一步despike的data节省空间
! rm pb01*

### 3. step3: distortion correction  
  
   因为磁场不均匀的关系，图像会出现畸变。现在来做矫正。

In [ ]:
# 首先找到
for fwd,rev in zip(fwdfiles, revfiles):
        # create median datasets from forward and reverse time series
        ! 3dTstat -median -prefix rm.blip.med.fwd.r{fwd} pb02.{subj}.r{fwd}.tshift+orig
        ! 3dTstat -median -prefix rm.blip.med.rev.r{rev} pb02.{subj}.r{rev}.tshift+orig
        # automask the median datasets 
        ! 3dAutomask -apply_prefix rm.blip.med.masked.fwd.r{fwd} rm.blip.med.fwd.r{fwd}+orig
        ! 3dAutomask -apply_prefix rm.blip.med.masked.rev.r{rev} rm.blip.med.rev.r{rev}+orig
        # compute the midpoint warp between the median datasets
        ! 3dQwarp -plusminus -pmNAMES Rev.r{rev} Fwd.r{fwd}                           \
                -pblur 0.05 0.05 -blur -1 -1                          \
                -noweight -minpatch 9                                 \
                -noXdis -noZdis                                       \
                -source rm.blip.med.masked.rev.r{rev}+orig                   \
                -base   rm.blip.med.masked.fwd.r{fwd}+orig                   \
                -prefix blip_warp
        
        ! 3dNwarpApply -quintic -nwarp blip_warp_Fwd.r{fwd}_WARP+orig        \
                -source rm.blip.med.masked.fwd.r{fwd}+orig               \
                -prefix rm.blip.med.masked.fwd.post.r{fwd}

        ! 3dNwarpApply -quintic -nwarp blip_warp_Rev.r{rev}_WARP+orig        \
                -source rm.blip.med.masked.rev.r{rev}+orig               \
                -prefix rm.blip.med.masked.rev.post.r{rev}
        
        # 删掉多余文件
        ! rm blip_warp_Fwd.r{fwd}+orig* blip_warp_Rev.r{rev}+orig* 

        # 修改个名字以便以后的操作
        ! 3dcopy blip_warp_Fwd.r{fwd}_WARP+orig blip_warp.r{fwd}_WARP+orig
        ! 3dcopy blip_warp_Rev.r{rev}_WARP+orig blip_warp.r{rev}_WARP+orig

        ! rm blip_warp_Fwd.r{fwd}+orig* blip_warp_Rev.r{rev}+orig* blip_warp_Fwd.r{fwd}_WARP+orig* blip_warp_Rev.r{rev}_WARP+orig* 

# for QC check，我们生成一个校正前和矫正后的平均EPI数据来作为distortion correction的效果证明
! 3dMean -prefix blip_pre_fwd.epi_mean.nii.gz rm.blip.med.masked.fwd.r*
! 3dMean -prefix blip_pre_rev.epi_mean.nii.gz rm.blip.med.masked.rev.r*
! 3dMean -prefix blip_post_fwd.epi_mean.nii.gz rm.blip.med.masked.fwd.post.r*
! 3dMean -prefix blip_post_rev.epi_mean.nii.gz rm.blip.med.masked.rev.post.r*

# remove redundent file
! rm rm*

这一步完成之后，可以手动打开
* blip_pre_fwd.epi_mean.nii.gz
* blip_pre_rev.epi_mean.nii.gz
* blip_post_fwd.epi_mean.nii.gz
* blip_post_rev.epi_mean.nii.gz

这四个文件也记录了对侧矫正的效果，只要有矫正效果，没有什么大问题即可。

然后把所有的run都做correction

In [ ]:
for run in runstr:
    ! 3dNwarpApply -quintic -nwarp blip_warp.r{run}_WARP+orig      \
            -source pb02.{subj}.r{run}.tshift+orig         \
            -prefix pb03.{subj}.r{run}.blip

### 4. step 4: motion correction  
  
    首先做motion correction。但是我们并不用其结果，主要是得到mc的线性转换矩阵

In [ ]:
# ================================= volreg =================================
base = 'vr_base_min_outlier+orig' #在前面生成

# align each dset to base volume, then we separate whether we want to further align epi to anat
for run in runstr:
    # register each volume to the base image
    ! 3dvolreg -verbose -zpad 1 -base {base} \
             -1Dfile dfile.r{run}.1D \
             -cubic \
             -1Dmatrix_save mat.r{run}.vr.aff12.1D \
             -prefix rm.epi.nomask.r{run} pb03.{subj}.r{run}.blip+orig

    # create an all-1 dataset to mask the extents of the warp
    ! 3dcalc -overwrite -a pb03.{subj}.r{run}.blip+orig -expr 1 -prefix rm.epi.all1

    # warp the all-1 dataset for extents masking
    ! 3dAllineate -base {base} \
                -input rm.epi.all1+orig \
                -1Dmatrix_apply mat.r{run}.vr.aff12.1D \
                -final NN -quiet \
                -prefix rm.epi.1.r{run}

    # make an extents intersection mask of this run across time domain
    # this makes a mask that all volumes in this run have valid numbers
    ! 3dTstat -min -prefix rm.epi.min.r{run} rm.epi.1.r{run}+orig

计算更多的头动数值，以后可以用来GLM中的regressor

In [ ]:
# --------------- deal with motion parameter -------------------
# we take dmeaned motion (6), demean motion derivative (6) and their squares (12)
# 合并 motion params
import numpy as np
import matplotlib.pyplot as plt

! cat dfile.r*.1D > dfile_rall.1D
# 计算去均值的motion parameters (for use in regression)
! 1d_tool.py -infile dfile_rall.1D -set_nruns {nRuns} -demean -write motion_demean.1D
# 计算一阶导数 (just to have)
! 1d_tool.py -infile dfile_rall.1D -set_nruns {nRuns} -derivative -demean -write motion_deriv.1D
# calculate the square of demean and derivative of motion parameters (just to have)
# for resting preproc, we usually have 24 motion regressor (6motion+6deriv+12 their square)
np.savetxt('motion_demeansq.1D', np.loadtxt('motion_demean.1D')**2)
np.savetxt('motion_derivsq.1D', np.loadtxt('motion_deriv.1D')**2)
# create censor file motion_${subj}_censor.1D, for censoring motion
! 1d_tool.py -infile dfile_rall.1D -set_nruns {nRuns} \
   -show_censor_count -censor_prev_TR \
   -censor_motion {motion_censor} motion_{subj}

# Estimate the motion parameter after motion correction, we can check the
# effects of MC
! mkdir mc
for run in runstr:
    ! 3dvolreg -verbose -zpad 1 -base {base} \
             -1Dfile dfile.r{run}_pos.1D \
             rm.epi.nomask.r{run}+orig
    ! rm volreg+orig*

# make a single file of motion params

! cat dfile.r*_pos.1D > dfile_rall_pos.1D

# make figure pre mc
dfile_pre = np.loadtxt('dfile_rall.1D')
plot(range(dfile_pre.shape[0]), dfile_pre[:,:3], color=['C0','C1','C2'], label=['roll(IS)','pitch(RL)','yaw(AP)'])
plt.legend();plt.xlabel('time points');plt.ylabel('mm');
plt.savefig('rots_pre.pdf');plt.close('all')
plot(range(dfile_pre.shape[0]), dfile_pre[:,3:], color=['C3','C4','C5'], label=['dS','dL','dP'])
plt.legend();plt.xlabel('time points (TR)');plt.ylabel('mm');
plt.savefig('tran_pre.pdf');plt.close('all')
# make figure pos mc
dfile_pos = np.loadtxt('dfile_rall_pos.1D')
plot(range(dfile_pos.shape[0]), dfile_pos[:,:3], color=['C0','C1','C2'], label=['roll(IS)','pitch(RL)','yaw(AP)'])
plt.legend();plt.xlabel('time points');plt.ylabel('mm');
plt.savefig('rots_pos.pdf');plt.close('all')
plot(range(dfile_pos.shape[0]), dfile_pos[:,3:], color=['C3','C4','C5'], label=['dS','dL','dP'])
plt.legend();plt.xlabel('time points (TR)');plt.ylabel('mm');
plt.savefig('tran_pos.pdf');plt.close('all')

# move file to directory

! mv rots*.pdf tran*.pdf dfile.r*_pos.1D dfile_r*_pos.1D mc/

# note TRs that were not censored, note ktrs here is a str
ktrs = unix_wrapper(f'1d_tool.py -infile motion_{subj}_censor.1D -show_trs_uncensored encoded', wantreturn=True, verbose=0)

# ----------------------------------------
# create the extents mask: mask_epi_extents+orig and apply the task
# (this is a mask of voxels that have valid data at every TR,
# there might be some pixel out of extents during mc)
! 3dMean -datum short -prefix rm.epi.mean rm.epi.min.r*.HEAD
! 3dcalc -a rm.epi.mean+orig -expr "step(a-0.999)" -prefix mask_epi_extents
# and apply the extents mask to the EPI data
# (delete any time series with missing data)
for run in runstr:
    ! 3dcalc -a rm.epi.nomask.r{run}+orig -b mask_epi_extents+orig \
           -expr "a*b" -prefix pb04.{subj}.r{run}.volreg
! rm -f rm.*  # rm.epi.nomask are big files, remove them
! rm -f {base}* 

# calculate a mean epi volume for next step anat epi registration
! 3dMean -prefix rm.epi_mean.nii.gz pb04.{subj}.r*.volreg+orig.HEAD
! 3dTstat -prefix epi_mean.nii.gz rm.epi_mean.nii.gz
! rm rm* mask_epi_extents+orig*

# 删掉上一步distortion correction的数据，节省空间
! rm pb03*

### 5. step 5: anat2epi  
  
    这一步非常重要，是前面处理好的功能像和结构像进行配准，配准结果一定要手动检查

In [ ]:
# 首先把用来配准的epi像make a copy
! 3dcopy epi_mean.nii.gz epi_mean_tmp.nii.gz

# 关键步骤。我们是把结构像配准到功能像，但是我们得到的转换矩阵是从相反的，功能像到结构像，这个矩阵可以后面用来转化epi数据
# 
! align_epi_anat.py -anat2epi -anat {t1.str} \
-save_skullstrip -suffix _al_junk \
-epi epi_mean_tmp.nii.gz -epi_base 0 \
-epi_strip 3dAutomask \
-volreg off -tshift off

# 生成nifti文件，方便我们在fsleyes上面查看配准效果
! 3dcopy {subj}_SurfVol_al_junk+orig {subj}_SurfVol_al_junk.nii.gz

完成上面一步之后，要打开afni仔细检查配准的结果，其中underlay选择 ***CN003_SurfVol_al_junk+orig***,  overlay选择 ***epi_mean_tmp+orig***，然后检查两者是否一致。  
如果不匹配，则需要重新进行上面的过程，在开始之前，需要先删除上面产生的文件。可以做  
    ! rm {subj}_SurfVol_al* {subj}_SurfVol_ns*
确认配准没有问题之后，进行下一步操作

In [ ]:
# note that we align anat 2 epi, however, this xfm is from epi space to anat space
# note that this xfm is compatible with LPS+ space, which is the default space in AFNI
! mv {t1.strnosuffix[:-5]}_al_junk_mat.aff12.1D mat.anat2epi.aff12.1D
# create an skull-striped anat_final dataset, aligned with stats
! 3dcopy {t1.strnosuffix[:-5]}_ns+orig anat_final.{subj}.nii.gz
! rm {t1.strnosuffix[:-5]}_ns+orig*
# rewrite the name of aligned anat
! 3dcopy {t1.strnosuffix[:-5]}_al_junk+orig anat2epi.{subj}.nii.gz
! rm {t1.strnosuffix[:-5]}_al_junk+orig*

# Invert xfm
! cat_matvec -ONELINE mat.anat2epi.aff12.1D -I > mat.epi2anat.aff12.1D

# warp the volreg base EPI dataset back to anat to make a final version
! 3dAllineate -base anat_final.{subj}.nii.gz \
            -input epi_mean.nii.gz \
            -1Dmatrix_apply mat.epi2anat.aff12.1D \
            -prefix epi2anat.{subj}.nii.gz

# Record final registration costs
! 3dAllineate -base epi2anat.{subj}.nii.gz -allcostX -input anat_final.{subj}.nii.gz > out.allcostX.txt

# Take the snapshots to show the quality of alignment
! @snapshot_volreg epi2anat.{subj}.nii.gz anat_final.{subj}.nii.gz
! @snapshot_volreg anat_final.{subj}.nii.gz epi2anat.{subj}.nii.gz
! @snapshot_volreg anat2epi.{subj}.nii.gz epi_mean.nii.gz
! @snapshot_volreg epi_mean.nii.gz anat2epi.{subj}.nii.gz

### 6. step 6: epi to individual anat
  
    现在我们有了epi到anat的配准，然后我们就可以把所有的epi的文件划到t1的空间。注意，这一步并不是从motion correction之后的文件(pb03.CN003.r0X.volreg+orig)。我们要从早distrotion correction之前的文件，把从DC-MC-epi2anat这三个转换一次性的做完

In [ ]:
resolution=2.5
# ======== Transform epi to match anat, add by RZ ===============
# In this step we need to concatenate Distortion(opt)+motion+affine transformations
# align each dset to base volume, then we separate whether we want to further align epi to anat
for run in runstr:
    # concatenate MC+aff transforms
    ! cat_matvec -ONELINE mat.anat2epi.aff12.1D -I mat.r{run}.vr.aff12.1D > mat.r{run}.warp.aff12.1D

    # we apply distortion correction + motion correction + anat2epi transformation
    cmd = f'3dNwarpApply -quintic -nwarp "mat.r{run}.warp.aff12.1D blip_warp.r{run}_WARP+orig" \
            -master {t1} -dxyz {resolution} \
            -source pb02.{subj}.r{run}.tshift+orig         \
            -prefix rm.epi.nomask.r{run}'
    unix_wrapper(cmd)

    # create an all-1 dataset to mask the extents of the warp
    ! 3dcalc -overwrite -a pb02.{subj}.r{run}.tshift+orig -expr 1 -prefix rm.epi.all1

    # we apply distortion correction + motion correction + anat2epi transformation
    cmd = f'3dNwarpApply -quintic -nwarp "mat.r{run}.warp.aff12.1D blip_warp.r{run}_WARP+orig" \
            -master {t1} -dxyz {resolution} \
            -ainterp NN -quiet \
            -source rm.epi.all1+orig \
            -prefix rm.epi.1.r{run}'
    unix_wrapper(cmd)

    # make an extents intersection mask of this run across time domain
    # this makes a mask that all volumes in this run have valid numbers
    ! 3dTstat -min -prefix rm.epi.min.r{run} rm.epi.1.r{run}+orig
    # below file is big, should be removed
    ! rm rm.epi.1.r{run}+orig*



# 变化之后，处理一下mask的问题
    # ----------------------------------------
# create the extents mask: mask_epi_extents+orig
# (this is a mask of voxels that have valid data at every TR,
# there might be some pixel out of extents during mc)
! 3dMean -datum short -prefix rm.epi.mean rm.epi.min.r*.HEAD
# and apply the extents mask to the EPI data
! 3dcalc -a rm.epi.mean+orig -expr "step(a-0.999)" -prefix mask_epi_extents

# 去掉mask之外的voxel的数据
for run in runstr:
    ! 3dcalc -a rm.epi.nomask.r{run}+orig -b mask_epi_extents+orig \
           -expr "a*b" -prefix pb05.{subj}.r{run}.al2anat
    # save to nifti format seems to reduce file size

# rm.epi.nomask are big files, remove them
! rm rm.*

### 7. step 7: epi volume 2 surface

   上一步我们已经把epi数据划到了和个体的结构像一个空间。很多时候我们需要做基于surface的数据分析。我们进一步把三维的epi数据插值划到surface上

In [ ]:
surface_dir = FREESURFER_HOME+f'/subjects/{subj}/SUMA'
# map volume data to the surface of each hemisphere
for hemi in ['lh', 'rh']:
    for  run in runstr:
        ! 3dVol2Surf -spec {surface_dir}/std.141.{subj}_{hemi}.spec   \
                   -sv {subj}_SurfVol+orig           \
                   -surf_A smoothwm                            \
                   -surf_B pial                                \
                   -f_index nodes                              \
                   -f_steps 10                                 \
                   -map_func ave                               \
                   -oob_value 0                                \
                   -grid_parent pb05.{subj}.r{run}.al2anat+orig   \
                   -out_niml pb05.{subj}.{hemi}.r{run}.surf.niml.dset

### 8. step 8: epi 2 MNI anat

In [ ]:
# ======== Transform epi to match anat, add by RZ ===============
# In this step we need to concatenate Distortion(opt)+motion+affine transformations
# align each dset to base volume, then we separate whether we want to further align epi to anat
for run in runstr:

    # we apply distortion correction + motion correction + anat2epi + nonlinear warp to MNI transformation
    # 注意我们这里是相反的顺序来concatenate的
    cmd = f'3dNwarpApply -quintic -nwarp "anatQQ.{subj}_WARP.nii anatQQ.{subj}.aff12.1D mat.r{run}.warp.aff12.1D blip_warp.r{run}_WARP+orig" \
            -master MNI152_2009_template_SSW.nii.gz \
            -dxyz {resolution} \
            -source pb02.{subj}.r{run}.tshift+orig \
            -prefix rm.epi.nomask.r{run}'
    unix_wrapper(cmd)

    # create an all-1 dataset to mask the extents of the warp
    ! 3dcalc -overwrite -a pb02.{subj}.r{run}.tshift+orig -expr 1 -prefix rm.epi.all1

    # we apply distortion correction + motion correction + anat2epi transformation
    cmd = f'3dNwarpApply -quintic -nwarp "anatQQ.{subj}_WARP.nii anatQQ.{subj}.aff12.1D mat.r{run}.warp.aff12.1D blip_warp.r{run}_WARP+orig" \
            -master MNI152_2009_template_SSW.nii.gz \
            -dxyz {resolution} \
            -source rm.epi.all1+orig         \
            -ainterp NN -quiet \
            -prefix rm.epi.1.r{run}'
    unix_wrapper(cmd)

    # make an extents intersection mask of this run across time domain
    # this makes a mask that all volumes in this run have valid numbers
    ! 3dTstat -min -prefix rm.epi.min.r{run} rm.epi.1.r{run}+tlrc
    # below file is big, should be removed
    ! rm rm.epi.1.r{run}+tlrc*



# 变化之后，处理mask的问题
# ----------------------------------------
# create the extents mask: mask_epi_extents+orig
# (this is a mask of voxels that have valid data at every TR,
# there might be some pixel out of extents during mc)
! 3dMean -datum short -prefix rm.epi.mean rm.epi.min.r*.HEAD
# and apply the extents mask to the EPI data
! 3dcalc -a rm.epi.mean+tlrc -expr "step(a-0.999)" -prefix mask_epi_2mni_extents

# 去掉mask之外的voxel的数据
for run in runstr:
    ! 3dcalc -a rm.epi.nomask.r{run}+tlrc -b mask_epi_2mni_extents+tlrc \
           -expr "a*b" -prefix pb06.{subj}.r{run}.al2mni
    # save to nifti format seems to reduce file size

# rm.epi.nomask are big files, remove them
! rm rm.*


In [ ]:
# 补充步骤，用于进行smooth操作，  2022.7.3添加
# 目的是对上一步step8所生成的pb06进行smooth   生成pb06s
# 再对上上一步step7所生成的pb05***surf.niml文件进行smooth  生成pb05s*rh  和 pb05s*lh
# 再对上上一步step7所生成的pb05.subj.r0*.al2anat+orig文件进行smooth  生成pb05s*.subj.r0*.blur+al2anat+orig

for run in runstr:
    ! 3dmerge -1blur_fwhm 4.0 -doall -prefix pb06s.{subj}.r{run}.blur+al2mni+tlrc \
        pb06.{subj}.r{run}.al2mni+tlrc

for run in runstr:
    ! 3dmerge -1blur_fwhm 4.0 -doall -prefix pb05s.{subj}.lh.r{run}.blur.surf.niml.dset \
        pb05.{subj}.lh.r{run}.surf.niml.dset

for run in runstr:
    ! 3dmerge -1blur_fwhm 4.0 -doall -prefix pb05s.{subj}.rh.r{run}.blur.surf.niml.dset \
        pb05.{subj}.rh.r{run}.surf.niml.dset

#以下代码可以不用跑
for run in runstr:
    ! 3dmerge -1blur_fwhm 4.0 -doall -prefix pb05s.{subj}.r{run}.blur+al2anat+orig \
        pb05.{subj}.r{run}.al2anat+orig


'''  
#afni官网上的对于surface数据进行smooth的代码

foreach hemi ( lh rh )
    foreach run ( $runs )
        # to save time, estimate blur parameters only once
        if ( ! -f surf.smooth.params.1D ) then
            SurfSmooth -spec $surface_dir/std.60.FT_${hemi}.spec         \
                       -surf_A smoothwm                                  \
                       -input pb03.$subj.$hemi.r$run.surf.niml.dset      \
                       -met HEAT_07                                      \
                       -target_fwhm 6.0                                  \
                       -blurmaster pb03.$subj.$hemi.r$run.surf.niml.dset \
                       -detrend_master                                   \
                       -output pb04.$subj.$hemi.r$run.blur.niml.dset     \
                       | tee surf.smooth.params.1D
        else
            set params = `1dcat surf.smooth.params.1D`
            SurfSmooth -spec $surface_dir/std.60.FT_${hemi}.spec         \
                       -surf_A smoothwm                                  \
                       -input pb03.$subj.$hemi.r$run.surf.niml.dset      \
                       -met HEAT_07                                      \
                       -Niter $params[1]                                 \
                       -sigma $params[2]                                 \
                       -output pb04.$subj.$hemi.r$run.blur.niml.dset
        endif
    end
end
'''

### 9. Step 9: 创造一个所有epi和mni模板共有的mask

In [ ]:
# ====================== Mask =================================
# Create 'full_mask' dataset (union mask)
for run in runstr:
    ! 3dAutomask -prefix rm.mask_r{run}.nii.gz pb06.{subj}.r{run}.al2mni+tlrc

# 创造一个所有功能像的run合起来的mask
! 3dmask_tool -inputs rm.mask_r*.nii.gz -union -prefix full_mask.{subj}.nii.gz
! rm rm.mask*

# ---- create subject anatomy mask, mask_anat.$subj+orig ----
#      (resampled from aligned anat)
# 把功能像resample到上面mask的分辨率
! 3dresample -master full_mask.{subj}.nii.gz -input \
           MNI152_2009_template_SSW.nii.gz -prefix rm.resam.anat.nii.gz
# convert to binary anat mask; fill gaps and holes
! 3dmask_tool -dilate_input 5 -5 -fill_holes -input rm.resam.anat.nii.gz \
            -prefix mask_anat.{subj}.nii.gz

# 结合功能像和结构像的mask
# compute tighter EPI mask by intersecting with anat mask
! 3dmask_tool -input full_mask.{subj}.nii.gz mask_anat.{subj}.nii.gz \
            -inter -prefix mask_epi_anat.{subj}.nii.gz
# note Dice coefficient of masks, as well
! 3ddot -dodice full_mask.{subj}.nii.gz mask_anat.{subj}.nii.gz > out.mask_ae_dice.txt

# 删掉多余文件
! rm rm*

### 10. Step 10: scale
    原始EPI图像的数据可能很大，从几百到4000不等。但是我们一般不考虑这个绝对强度，考虑的是信号增长的比例。所以把数据scale一下, 让所有的voxel的时间序列均值为100

In [ ]:
# scale each voxel time series to have a mean of 100
# (be sure no negatives creep in)
# (subject to a range of [0,200])

# scale volume data
for run in runstr:
    ! 3dTstat -prefix rm.mean_r{run} pb06.{subj}.r{run}.al2mni+tlrc
    ! 3dcalc -a pb06.{subj}.r{run}.al2mni+tlrc -b rm.mean_r{run}+tlrc \
           -c mask_epi_2mni_extents+tlrc \
           -expr "c * min(200, a/b*100)*step(a)*step(b)" \
           -prefix pb07.{subj}.r{run}.scale

# combine all datasets into one
! 3dTcat -prefix all_runs.{subj}.nii.gz pb07.{subj}.r*.scale+tlrc*
# remove redundant file
! rm rm.mean*

# 再scale surface data
for hemi in ['lh', 'rh']:
    for run in runstr:
       ! 3dTstat -prefix rm.{hemi}.mean_r{run}.niml.dset    \
            pb05.{subj}.{hemi}.r{run}.surf.niml.dset
       ! 3dcalc -a pb05.{subj}.{hemi}.r{run}.surf.niml.dset  \
               -b rm.{hemi}.mean_r{run}.niml.dset          \
               -expr 'min(200, a/b*100)*step(a)*step(b)' \
               -prefix pb07.{subj}.{hemi}.r{run}.scale.niml.dset

# remove redundant file
! rm rm.*mean* 

到这一步，基本上最主要的预处理就完结了。下一步可以进一步针对surface数据跑GLM，因为afni的surface都是在freesurfer的模板上对其且标准化的，所以surface数据可以直接跑group analysis。也可以在已经划到MNI space上的volume数据跑GLM。以后在volume上跑GLM的结果可以直接用MNI152的T1像来可视化

## funcpp03

### 1 step1
先跑一个GLM，把所有头动信息回归掉，然后计算一系列诊断信息

In [1]:
# ================================ regress =================================
# compute de-meaned motion parameters (for use in regression)
# in regress (detrend step)

# ------------------------------
# run the regression analysis. Note that stats results should be written as afni format

! 3dDeconvolve -input pb07.{subj}.r*.scale+tlrc.HEAD \
    -mask mask_epi_anat.{subj}.nii.gz \
    -censor motion_{subj}_censor.1D \
    -ortvec motion_demean.1D mot_demean \
    -polort 2 \
    -num_stimts 0 \
    -rout \
    -fout -tout -x1D X.xmat.1D -xjpeg X.jpg \
    -x1D_uncensored X.nocensor.xmat.1D \
    -fitts motpoly.fitts.{subj} \
    -errts motpoly.errts.{subj} \
    -bucket motpoly.stats.{subj}

# should add save a script here

# -- use 3dTproject to project out regression matrix --
! 3dTproject -polort 0 -input pb07.{subj}.r*.scale+tlrc.HEAD \
           -censor motion_{subj}_censor.1D -cenmode ZERO \
           -ort X.nocensor.xmat.1D -prefix motpoly.errts.{subj}.tproject
# 3dTproject can also supply '-passband 0.01, 0.08' to band pass time seriest

! rm -f motpoly.stats* motpoly.fitts* # remove some large files

'3dDeconvolve' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'3dTproject' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���
'rm' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


### 2. step2 计算temporal sigmal-to-noise ratio

In [ ]:
# --------------------------------------------------
# create a temporal signal to noise ratio dataset
#    signal: if 'scale' block, mean should be 100
#    noise : compute standard deviation of errts
unix_wrapper(f'3dTstat -mean -prefix rm.signal.all.nii.gz all_runs.{subj}.nii.gz"[{ktrs}]"')
unix_wrapper(f'3dTstat -stdev -prefix rm.noise.all.nii.gz motpoly.errts.{subj}.tproject+tlrc"[{ktrs}]"')
! 3dcalc -a rm.signal.all.nii.gz \
       -b rm.noise.all.nii.gz \
       -c full_mask.{subj}.nii.gz \
       -expr "c*a/b" -prefix TSNR.{subj}.nii.gz

# remove redundant files
! rm rm.signal.all* rm.noise.all*
! rm motpoly.errts* X*

### 3. step3: clean up

In [ ]:
# ========================== auto block: finalize ==========================
# Remove temporary files
! rm -f rm.* # finally remove some left-over files

# We can further remove despike, tshift and volreg data at the end volreg are big
! rm -f all_runs*
! rm anatQQ* MNI152_2009_template_SSW.nii.gz
! rm pb02* pb04* pb05* 
! rm epi_mean_tmp*

# organize data


# ==================== preprocessing done ==================================
# return to previous directory
os.chdir(cwd)
os.environ['SHELL'] = orig_shell     # switch back to the original shell
print(f'\n=============== Preprocessing started: {start_time} ================\n')
print(f'\n=============== Preprocessing finished: {gettimestr("full")} ================\n')